# Puzzle generation

In [11]:
import numpy as np
import json
from typing import List, Tuple

NUMBER_OF_EASY = 10
NUMBER_OF_MEDIUM = 10
NUMBER_OF_HARD = 10

def extract_constraints(grid: np.ndarray) -> Tuple[List[List[int]], List[List[int]]]:
    def get_line_constraints(line):
        constraints = []
        count = 0
        for val in line:
            if val == 1:
                count += 1
            elif count > 0:
                constraints.append(count)
                count = 0
        if count > 0:
            constraints.append(count)
        return constraints or [0]

    row_constraints = [get_line_constraints(row) for row in grid]
    col_constraints = [get_line_constraints(col) for col in grid.T]
    return row_constraints, col_constraints

def generate_puzzles(size: int, count: int) -> List[dict]:
    puzzles = []
    while len(puzzles) < count:
        solution = np.random.randint(0, 2, (size, size))
        row_cons, col_cons = extract_constraints(solution)
        puzzles.append({
            "row": row_cons,
            "col": col_cons,
            "solution": solution.tolist()
        })
    return puzzles

def save_to_file(puzzles: List[dict], filename: str):
    with open(filename, 'w') as f:
        for puzzle in puzzles:
            f.write(json.dumps(puzzle) + '\n')


save_to_file(generate_puzzles(5, NUMBER_OF_EASY), "small.txt")
save_to_file(generate_puzzles(10, NUMBER_OF_MEDIUM), "medium.txt")
save_to_file(generate_puzzles(15, NUMBER_OF_HARD), "large.txt")


# Code for getting solution statistics

In [15]:
import numpy as np
from numpy.typing import NDArray
import json

def check(grid: np.ndarray, row_constraints, col_constraints) -> bool:
    def get_blocks(line) -> list[int]:
        return [sum(g) for g in np.split(line, np.where(line == 0)[0]) if sum(g) > 0]

    for i, row in enumerate(grid):
        if get_blocks(row) != row_constraints[i]:
            return False
    for j, col in enumerate(grid.T):
        if get_blocks(col) != col_constraints[j]:
            return False
    return True

def get_blocks(line: np.ndarray) -> list[int]:
    blocks = []
    count = 0
    for cell in line:
        if cell == 1:
            count += 1
        elif count:
            blocks.append(count)
            count = 0
    if count:
        blocks.append(count)
    return blocks

def puzzle_accuracy(grid: NDArray[np.int_], row_constraints: list[list[int]], col_constraints: list[list[int]]) -> float:
    """Proportion of rows and columns that match their constraints."""
    grid_size = grid.shape[0]
    correct_rows = sum(get_blocks(grid[i]) == row_constraints[i] for i in range(grid_size))
    correct_cols = sum(get_blocks(grid[:, j]) == col_constraints[j] for j in range(grid_size))
    return (correct_rows + correct_cols) / (2 * grid_size)

def constraints_accuracy(grid: NDArray[np.int_], row_constraints: list[list[int]], col_constraints: list[list[int]]) -> float:
    """Proportion of individual constraint elements (numbers in row/col specs) that match."""
    matched = 0
    total = 0

    for i in range(grid.shape[0]):
        actual = get_blocks(grid[i])
        expected = row_constraints[i]
        total += len(expected)
        matched += sum(1 for a, b in zip(actual, expected) if a == b)

    for j in range(grid.shape[1]):
        actual = get_blocks(grid[:, j])
        expected = col_constraints[j]
        total += len(expected)
        matched += sum(1 for a, b in zip(actual, expected) if a == b)

    if total == 0:
        return 1.0  # no constraints to match
    return matched / total


def save_result(results, filename: str):
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)


# Genetic Algorithm

In [16]:
import random
import numpy as np
from numpy.typing import NDArray
import time
import json

# Set global vars for the solver
mutation_rate = 0.01
generations = 100
population_size = 100

# Load puzzles
with open("small.txt") as f:
    small = [json.loads(line) for line in f]
with open("medium.txt") as f:
    medium = [json.loads(line) for line in f]
with open("large.txt") as f:
    large = [json.loads(line) for line in f]

results = {"small": [], "medium": [], "large": []}

def create_individual() -> list[int]:
    """Generate a random individual (binary grid representation)."""
    return [random.randint(0, 1) for _ in range(grid_size**2)]

def evaluate(individual: list[int]) -> int:
    """Evaluate the fitness of an individual based on row and column constraints."""
    grid = np.array(individual).reshape((grid_size, grid_size))
    score = 0  # Higher score means a better match to constraints

    # Evaluate row constraints
    for i, row in enumerate(grid):
        blocks = [sum(g) for g in np.split(row, np.where(row == 0)[0]) if sum(g) > 0]
        if blocks == row_constraints[i]:  
            score += 1  

    # Evaluate column constraints
    for j, col in enumerate(grid.T):  # Transpose to iterate over columns
        blocks = [sum(g) for g in np.split(col, np.where(col == 0)[0]) if sum(g) > 0]
        if blocks == col_constraints[j]:  
            score += 1  

    return score  # Higher score means better constraint satisfaction

def select(population: list[list[int]]) -> list[list[int]]:
    """Select the top-performing individuals for reproduction."""
    population.sort(key=evaluate, reverse=True)  # Sort by fitness (descending order)
    return population[:population_size // 2]  # Retain the top 50% of individuals

def crossover(parent1: list[int], parent2: list[int]) -> tuple[list[int], list[int]]:
    """Perform single-point crossover between two parents."""
    point = random.randint(0, grid_size**2 - 1)  # Choose a crossover point
    child1 = parent1[:point] + parent2[point:]  
    child2 = parent2[:point] + parent1[point:]  
    return child1, child2

def mutate(individual: list[int]) -> None:
    """Apply mutation to an individual with a certain probability."""
    for i in range(len(individual)):
        if random.random() < mutation_rate:  
            individual[i] = 1 - individual[i]  # Flip the bit (0 → 1, 1 → 0)

def genetic_algorithm() -> NDArray[np.long]:
    """Solve the Picross puzzle using a genetic algorithm."""
    population = [create_individual() for _ in range(population_size)]

    for generation in range(generations):
        population = select(population)  # Select the fittest individuals

        # If an individual satisfies all constraints, terminate early
        if evaluate(population[0]) == grid_size * 2:
            break

        next_generation = population.copy()

        # Generate offspring until the population size is restored
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)  # Select two parents
            child1, child2 = crossover(parent1, parent2)  # Perform crossover
            mutate(child1)  # Apply mutation
            mutate(child2)  
            next_generation.extend([child1, child2])  # Add offspring to the population
        
        population = next_generation[:population_size]  # Ensure population size remains constant
        
        best_individual = max(population, key=evaluate)  # Track the best solution found

    # Display the best solution found
    best_solution = np.array(best_individual).reshape((grid_size, grid_size))
    
    return best_solution

# Run the genetic algorithm to solve the Picross puzzle
for size_name, puzzles in [("small", small), ("medium", medium), ("large", large)]:
    for puzzle in puzzles:
        row_constraints = puzzle["row"]
        col_constraints = puzzle["col"]
        grid_size = len(row_constraints)

        start = time.time()
        grid = genetic_algorithm()
        end = time.time()
        print(end - start)
        
        correct = check(grid, row_constraints, col_constraints)
        p_acc = puzzle_accuracy(grid, row_constraints, col_constraints)
        c_acc = constraints_accuracy(grid, row_constraints, col_constraints)

        results[size_name].append({
            "solved": correct,
            "time": end - start,
            "puzzle_accuracy": p_acc,
            "constraints_accuracy": c_acc
        })

# Save results
save_result(results, "GA-results.json")


2.457031488418579
2.4266605377197266
0.6259183883666992
2.3171470165252686
0.4116501808166504
0.4122297763824463
2.057175397872925
2.1018645763397217
0.15682339668273926
2.322035074234009
6.426980972290039
5.687209367752075
5.964537858963013
6.145095586776733
5.8329901695251465
6.0331130027771
5.986798524856567
5.854884147644043
5.908690690994263
5.817958116531372
11.576941967010498
10.9504976272583
11.526744842529297
11.48019528388977
12.675728559494019
11.40392017364502
11.256325483322144
11.528033971786499
12.641906023025513
11.19859766960144


# Simulated Annealing

In [19]:
import random
import numpy as np
from numpy.typing import NDArray
import time
import json

# Set global vars for the solver
initial_temperature = 10.0
cooling_rate = 0.999
iterations = 10000

# Load puzzles
with open("small.txt") as f:
    small = [json.loads(line) for line in f]
with open("medium.txt") as f:
    medium = [json.loads(line) for line in f]
with open("large.txt") as f:
    large = [json.loads(line) for line in f]

results = {"small": [], "medium": [], "large": []}

def evaluate(grid: NDArray[np.long]) -> int:
    """
    Calculate the fitness score of a grid based on row and column constraints.
    A higher score indicates a better match to the constraints.
    """
    score = 0  # Initialize score
    
    # Evaluate row constraints
    for i, row in enumerate(grid):
        # Find contiguous blocks of 1s in the row
        blocks = [sum(g) for g in np.split(row, np.where(row == 0)[0]) if sum(g) > 0]
        if blocks == row_constraints[i]:  
            score += 1  

    # Evaluate column constraints
    for j, col in enumerate(grid.T):  # Transpose to iterate over columns
        blocks = [sum(g) for g in np.split(col, np.where(col == 0)[0]) if sum(g) > 0]
        if blocks == col_constraints[j]:  
            score += 1  

    return score  # Higher score means better constraint satisfaction

def mutate(grid: NDArray[np.long]) -> NDArray[np.long]:
    """
    Create a new grid by flipping a random cell (mutating the grid).
    """
    new_grid = grid.copy()  # Copy the current grid
    i, j = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)  # Select a random cell
    new_grid[i, j] = 1 - new_grid[i, j]  # Flip the bit (0 → 1, 1 → 0)
    return new_grid

def simulated_annealing() -> NDArray[np.long]:
    """
    Solve the Picross puzzle using the simulated annealing algorithm.
    """
    # Initialize a random grid with binary values (0 or 1)
    current_grid = np.random.randint(0, 2, (grid_size, grid_size))
    current_score = evaluate(current_grid)  # Compute its initial fitness score
    temperature = initial_temperature  # Set the initial temperature
    
    # Keep track of the best solution found
    best_grid, best_score = current_grid, current_score

    for step in range(iterations):
        # Generate a new candidate solution by mutating the current grid
        new_grid = mutate(current_grid)
        new_score = evaluate(new_grid)

        # If the new grid is better, accept it unconditionally
        if new_score > current_score:
            current_grid, current_score = new_grid, new_score
        else:
            # Otherwise, accept with a probability determined by the temperature
            delta = new_score - current_score
            if random.random() < np.exp(delta / temperature):  
                current_grid, current_score = new_grid, new_score

        # Update the best solution if the new one is the best so far
        if new_score > best_score:
            best_grid, best_score = new_grid, new_score

        # Reduce the temperature over time
        temperature *= cooling_rate

        # Stop early if a perfect solution is found
        if best_score == grid_size * 2:
            break

    return best_grid

# Run the simulated annealing algorithm to solve the Picross puzzle
for size_name, puzzles in [("small", small), ("medium", medium), ("large", large)]:
    for puzzle in puzzles:
        row_constraints = puzzle["row"]
        col_constraints = puzzle["col"]
        grid_size = len(row_constraints)

        start = time.time()
        grid = simulated_annealing()
        end = time.time()
        print(end - start)
        correct = check(grid, row_constraints, col_constraints)
        p_acc = puzzle_accuracy(grid, row_constraints, col_constraints)
        c_acc = constraints_accuracy(grid, row_constraints, col_constraints)

        results[size_name].append({
            "solved": correct,
            "time": end - start,
            "puzzle_accuracy": p_acc,
            "constraints_accuracy": c_acc
        })

# Save results
save_result(results, "SA-results.json")


0.33307886123657227
0.38756585121154785
0.3683464527130127
0.4144022464752197
0.5384764671325684
0.38638782501220703
1.1111764907836914
1.073246955871582
1.0377464294433594
0.4425191879272461
2.8571043014526367
2.956803321838379
2.861356019973755
2.912015438079834
2.8502087593078613
2.9537391662597656
2.8742332458496094
2.8093721866607666
2.8586251735687256
2.831289529800415
5.6097142696380615
5.560453414916992
5.386080741882324
6.037994384765625
5.658936977386475
5.77406644821167
5.61034631729126
5.44935941696167
5.95748233795166
5.95815110206604


# Results Analysis

In [20]:
import json

def analyze(file: str):
    with open(file) as f:
        data = json.load(f)

    for size, records in data.items():
        total = len(records)
        solved = sum(1 for r in records if r["solved"])
        avg_time = sum(r["time"] for r in records) / max(total, 1)
        avg_puzzle_acc = sum(r.get("puzzle_accuracy", 0) for r in records) / max(total, 1)
        avg_constraint_acc = sum(r.get("constraints_accuracy", 0) for r in records) / max(total, 1)

        print(f"{file} - {size}:")
        print(f"  Accuracy: {solved / max(total, 1) * 100:.1f}%")
        print(f"  Avg puzzle accuracy: {avg_puzzle_acc * 100:.2f}%")
        print(f"  Avg constraint accuracy: {avg_constraint_acc * 100:.2f}%")
        print(f"  Avg time: {avg_time:.4f}s\n")

analyze("GA-results.json")
analyze("SA-results.json")

GA-results.json - small:
  Accuracy: 40.0%
  Avg puzzle accuracy: 88.00%
  Avg constraint accuracy: 90.36%
  Avg time: 1.5289s

GA-results.json - medium:
  Accuracy: 0.0%
  Avg puzzle accuracy: 51.00%
  Avg constraint accuracy: 64.20%
  Avg time: 5.9658s

GA-results.json - large:
  Accuracy: 0.0%
  Avg puzzle accuracy: 25.00%
  Avg constraint accuracy: 42.86%
  Avg time: 11.6239s

SA-results.json - small:
  Accuracy: 70.0%
  Avg puzzle accuracy: 95.00%
  Avg constraint accuracy: 96.52%
  Avg time: 0.6093s

SA-results.json - medium:
  Accuracy: 0.0%
  Avg puzzle accuracy: 76.50%
  Avg constraint accuracy: 82.92%
  Avg time: 2.8765s

SA-results.json - large:
  Accuracy: 0.0%
  Avg puzzle accuracy: 41.00%
  Avg constraint accuracy: 58.84%
  Avg time: 5.7003s

